# Twitter Sentiment Analysis - Understanding Emotions at Scale

**Author:** Anik Tahabilder  
**Project:** 7 of 22 - Kaggle ML Portfolio  
**Dataset:** Sentiment140 (1.6 Million Tweets)  
**Difficulty:** 5/10 | **Resume Value:** 7/10 | **Learning Value:** 8/10 | **Impact:** 8/10

---

## What is Sentiment Analysis?

**Sentiment Analysis** (also called opinion mining) is the process of computationally determining whether a piece of text expresses a **positive**, **negative**, or **neutral** opinion.

In this notebook, we'll build models that can automatically detect if a tweet expresses:
- 😊 **Positive sentiment** (happy, excited, satisfied)
- 😞 **Negative sentiment** (angry, sad, disappointed)

### Real-World Applications:

| Industry | Application | Example |
|----------|-------------|----------|
| **Social Media** | Monitor brand reputation | Track customer opinions about products |
| **Customer Service** | Prioritize urgent issues | Identify angry customers for immediate response |
| **Finance** | Market sentiment analysis | Predict stock movements from news/tweets |
| **Politics** | Public opinion polling | Gauge voter sentiment during elections |
| **Product Development** | Feature feedback | Understand what users love/hate |
| **E-commerce** | Review analysis | Summarize thousands of product reviews |

### Why is Sentiment Analysis Challenging?

Natural language is complex and ambiguous:
- **Sarcasm**: "Great, another bug!" (negative despite "great")
- **Context**: "This movie is sick!" (positive in slang, negative literally)
- **Negation**: "not bad" (positive) vs "bad" (negative)
- **Emojis**: 😂 can mean funny OR mocking
- **Slang & Abbreviations**: "omg so lit" vs formal language

---

## Table of Contents

1. [Part 1: Understanding Sentiment Analysis & NLP](#part1)
2. [Part 2: Setup and Data Loading](#part2)
3. [Part 3: Exploratory Data Analysis](#part3)
4. [Part 4: Text Preprocessing Pipeline](#part4)
5. [Part 5: Feature Extraction Methods](#part5)
6. [Part 6: Classical Machine Learning Models](#part6)
7. [Part 7: Deep Learning with LSTM](#part7)
8. [Part 8: Model Comparison](#part8)
9. [Part 9: Word Clouds & Visualization](#part9)
10. [Part 10: Predictions on New Tweets](#part10)
11. [Part 11: Summary and Key Takeaways](#part11)

---

<a id='part1'></a>
# Part 1: Understanding Sentiment Analysis & NLP

---

## 1.1 What is Natural Language Processing (NLP)?

**Natural Language Processing** is a branch of AI focused on enabling computers to understand, interpret, and generate human language.

### NLP Task Hierarchy:

| Task | Goal | Difficulty |
|------|------|------------|
| **Tokenization** | Split text into words | Easy |
| **Sentiment Analysis** | Classify emotion | Medium |
| **Named Entity Recognition** | Extract names, places, organizations | Medium-Hard |
| **Machine Translation** | Translate between languages | Hard |
| **Question Answering** | Answer questions about text | Very Hard |

### The NLP Pipeline:

```
Raw Text → Preprocessing → Feature Extraction → Model → Prediction
"I love this!"
    ↓                ↓                  ↓             ↓
Clean text     [love, this]      [0.2, 0.8, ...]   Positive!
```

## 1.2 Sentiment Analysis vs Other NLP Tasks

| Task Type | Input | Output | Example |
|-----------|-------|--------|----------|
| **Sentiment Analysis** | Text | Sentiment label | "I hate this" → Negative |
| **Topic Classification** | Text | Topic category | "iPhone 15 released" → Technology |
| **Spam Detection** | Email | Spam/Not Spam | "Win $1M now!" → Spam |
| **Text Generation** | Prompt | Generated text | "Once upon a time" → Story |

## 1.3 Our Approach

We'll build sentiment analysis models using:
1. **Classical ML**: Logistic Regression, Naive Bayes, SVM, Random Forest
2. **Deep Learning**: LSTM (Long Short-Term Memory) networks

Then compare their performance on 1.6 million tweets!

---

<a id='part2'></a>
# Part 2: Setup and Data Loading

---

## 2.1 Import Required Libraries

For sentiment analysis, we need:

| Library | Purpose |
|---------|----------|
| **pandas/numpy** | Data manipulation |
| **matplotlib/seaborn** | Visualization |
| **nltk** | Natural Language Toolkit (preprocessing) |
| **sklearn** | Classical ML algorithms |
| **tensorflow/keras** | Deep learning (LSTM) |
| **wordcloud** | Visualize word frequencies |
| **re** | Regular expressions (text cleaning) |

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Suppress TensorFlow warnings BEFORE importing
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Text processing
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Utilities
import time

# Download NLTK data
print("Downloading NLTK data...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt_tab', quiet=True)
print("NLTK data downloaded!")

# Display settings
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_colwidth', 100)
np.random.seed(42)
tf.random.set_seed(42)

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

# Check GPU
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"\nGPU Available: {len(gpus)} GPU(s)")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("\nNo GPU detected - using CPU")
except:
    print("\nUsing CPU for training")

## 2.2 Loading the Sentiment140 Dataset

### About Sentiment140:

- **Size**: 1.6 million tweets
- **Source**: Twitter API (2009)
- **Labels**: Binary (0 = negative, 4 = positive)
- **Emoticons removed**: To prevent models from cheating
- **Language**: English

### Dataset Structure:

| Column | Description | Example |
|--------|-------------|----------|
| **target** | Sentiment (0=negative, 4=positive) | 0 |
| **ids** | Tweet ID | 1467810369 |
| **date** | Timestamp | Mon Apr 06 22:19:45 PDT 2009 |
| **flag** | Query (NO_QUERY if none) | NO_QUERY |
| **user** | Username | scotthamilton |
| **text** | Tweet content | @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. |

In [ ]:
# Load the dataset
# Note: The dataset is encoded in latin-1, not UTF-8
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']

try:
    # For Kaggle environment
    df = pd.read_csv(
        '/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
        encoding='latin-1',
        names=column_names
    )
    print("Dataset loaded from Kaggle!")
except:
    # For local environment (if you have the file)
    df = pd.read_csv(
        'training.1600000.processed.noemoticon.csv',
        encoding='latin-1',
        names=column_names
    )
    print("Dataset loaded from local file!")

print("="*60)
print("SENTIMENT140 DATASET")
print("="*60)
print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Display first few tweets
print("First 10 tweets:")
df.head(10)

In [ ]:
# Dataset info
print("Dataset Information:")
print("="*60)
df.info()

print("\n" + "="*60)
print("Memory Usage:")
print(f"Total: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Check for missing values
print("Missing Values Check:")
print("="*60)
missing = df.isnull().sum()
print(missing)
print(f"\nTotal missing values: {missing.sum()}")
print("\nGreat! No missing values in the dataset.")

In [ ]:
# Convert target from (0, 4) to (0, 1) for easier interpretation
# 0 = negative, 4 = positive → 0 = negative, 1 = positive
df['sentiment'] = df['target'].map({0: 0, 4: 1})

# Verify the conversion
print("Sentiment Conversion:")
print("="*60)
print("Original target values:", df['target'].unique())
print("New sentiment values:", df['sentiment'].unique())
print("\n0 = Negative, 1 = Positive")

---

<a id='part3'></a>
# Part 3: Exploratory Data Analysis

---

## 3.1 Class Distribution

**Critical Question**: Is our dataset balanced?

An imbalanced dataset can cause:
- Model bias toward majority class
- Misleading accuracy metrics
- Poor performance on minority class

In [ ]:
# Class distribution
sentiment_counts = df['sentiment'].value_counts().sort_index()

print("Class Distribution:")
print("="*60)
print(sentiment_counts)
print(f"\nNegative tweets (0): {sentiment_counts[0]:,} ({sentiment_counts[0]/len(df)*100:.2f}%)")
print(f"Positive tweets (1): {sentiment_counts[1]:,} ({sentiment_counts[1]/len(df)*100:.2f}%)")
print("\nThis is a PERFECTLY BALANCED dataset!")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
colors = ['#FF6B6B', '#4ECDC4']
labels = ['Negative', 'Positive']
bars = axes[0].bar(labels, sentiment_counts.values, color=colors, edgecolor='black')
axes[0].set_ylabel('Count')
axes[0].set_title('Sentiment Distribution', fontweight='bold', fontsize=14)
for bar, val in zip(bars, sentiment_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10000,
                 f'{val:,}', ha='center', fontweight='bold', fontsize=12)

# Pie chart
axes[1].pie(sentiment_counts.values, labels=labels, autopct='%1.1f%%',
            colors=colors, explode=(0.02, 0.02), shadow=True,
            textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Sentiment Proportion', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

print("Perfect 50-50 split means no class imbalance to worry about!")

## 3.2 Tweet Length Analysis

Understanding tweet length helps us:
- Set appropriate max length for deep learning models
- Identify potential outliers
- Understand data characteristics

In [ ]:
# Calculate tweet lengths
df['text_length'] = df['text'].astype(str).apply(len)
df['word_count'] = df['text'].astype(str).apply(lambda x: len(x.split()))

# Statistics by sentiment
print("Tweet Length Statistics:")
print("="*60)
print("\nCharacter Length:")
print(df.groupby('sentiment')['text_length'].describe())
print("\nWord Count:")
print(df.groupby('sentiment')['word_count'].describe())

In [ ]:
# For speed, let's sample the data for visualization
# (Otherwise plotting 1.6M points takes forever)
sample_df = df.sample(n=50000, random_state=42)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Character length distribution
for sentiment, color, label in zip([0, 1], ['#FF6B6B', '#4ECDC4'], ['Negative', 'Positive']):
    subset = sample_df[sample_df['sentiment'] == sentiment]
    axes[0, 0].hist(subset['text_length'], bins=50, alpha=0.6, color=color, label=label, edgecolor='black')
axes[0, 0].set_xlabel('Character Length')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Tweet Character Length Distribution', fontweight='bold')
axes[0, 0].legend()

# Word count distribution
for sentiment, color, label in zip([0, 1], ['#FF6B6B', '#4ECDC4'], ['Negative', 'Positive']):
    subset = sample_df[sample_df['sentiment'] == sentiment]
    axes[0, 1].hist(subset['word_count'], bins=30, alpha=0.6, color=color, label=label, edgecolor='black')
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Tweet Word Count Distribution', fontweight='bold')
axes[0, 1].legend()

# Box plots
sample_df.boxplot(column='text_length', by='sentiment', ax=axes[1, 0])
axes[1, 0].set_title('Character Length by Sentiment', fontweight='bold')
axes[1, 0].set_xlabel('Sentiment (0=Negative, 1=Positive)')
axes[1, 0].set_ylabel('Character Length')

sample_df.boxplot(column='word_count', by='sentiment', ax=axes[1, 1])
axes[1, 1].set_title('Word Count by Sentiment', fontweight='bold')
axes[1, 1].set_xlabel('Sentiment (0=Negative, 1=Positive)')
axes[1, 1].set_ylabel('Word Count')

plt.suptitle('Tweet Length Analysis (50K Sample)', fontweight='bold', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- Most tweets are 100-140 characters (Twitter's old limit)")
print("- Average word count is around 15-20 words")
print("- Both sentiments have similar length distributions")

## 3.3 Sample Tweets from Each Sentiment

Let's look at actual examples to understand what we're working with.

In [ ]:
# Sample tweets from each sentiment
print("SAMPLE NEGATIVE TWEETS (sentiment = 0):")
print("="*60)
negative_samples = df[df['sentiment'] == 0]['text'].sample(10, random_state=42).values
for i, tweet in enumerate(negative_samples, 1):
    print(f"{i}. {tweet}")
    print()

print("\n" + "="*60)
print("SAMPLE POSITIVE TWEETS (sentiment = 1):")
print("="*60)
positive_samples = df[df['sentiment'] == 1]['text'].sample(10, random_state=42).values
for i, tweet in enumerate(positive_samples, 1):
    print(f"{i}. {tweet}")
    print()

## 3.4 Data Characteristics

What patterns do we notice in the raw tweets?

In [ ]:
# Analyze tweet characteristics
sample_analysis = df.sample(n=100000, random_state=42)

# Count tweets with specific patterns
patterns = {
    'URLs': sample_analysis['text'].str.contains('http', case=False, na=False).sum(),
    'Mentions (@)': sample_analysis['text'].str.contains('@', na=False).sum(),
    'Hashtags (#)': sample_analysis['text'].str.contains('#', na=False).sum(),
    'Numbers': sample_analysis['text'].str.contains(r'\d', na=False).sum(),
}

print("Tweet Characteristics (100K sample):")
print("="*60)
for pattern, count in patterns.items():
    percentage = (count / len(sample_analysis)) * 100
    print(f"{pattern:<20}: {count:>7,} ({percentage:>5.1f}%)")

print("\nWe'll need to clean these during preprocessing!")

---

<a id='part4'></a>
# Part 4: Text Preprocessing Pipeline

---

## 4.1 Why Preprocess Text?

Raw text is messy and inconsistent. Preprocessing helps by:

| Problem | Solution |
|---------|----------|
| **Case sensitivity** | "Happy" ≠ "happy" | Convert to lowercase |
| **URLs** | "http://bit.ly/abc" adds noise | Remove URLs |
| **Mentions** | "@user" varies by user | Remove @ mentions |
| **Special characters** | "!!!" doesn't add meaning | Remove punctuation |
| **Numbers** | Often not relevant | Remove digits |
| **Stop words** | "the", "is", "at" appear everywhere | Remove common words |
| **Word variations** | "running", "runs", "ran" | Stemming/Lemmatization |

## 4.2 Preprocessing Steps

Our pipeline will:
1. **Lowercase** - Standardize case
2. **Remove URLs** - Strip http:// links
3. **Remove mentions** - Strip @username
4. **Remove hashtags** - Strip # symbols (keep text)
5. **Remove special characters** - Keep only letters
6. **Remove numbers** - Strip digits
7. **Tokenize** - Split into words
8. **Remove stopwords** - Filter common words
9. **Stem/Lemmatize** - Reduce words to root form

## 4.3 Stopwords - What Are They?

**Stopwords** are common words that appear frequently but carry little meaning:
- Articles: "a", "an", "the"
- Pronouns: "I", "you", "he", "she"
- Prepositions: "in", "on", "at", "to"
- Conjunctions: "and", "but", "or"

Removing them:
- Reduces vocabulary size
- Focuses on meaningful words
- Speeds up training

**However**: For sentiment, some stopwords matter!
- "not" is critical for negation
- "very" intensifies sentiment

We'll use a custom stopword list.

In [ ]:
# Get NLTK stopwords
stop_words = set(stopwords.words('english'))

# Remove negation words from stopwords (they're important for sentiment!)
negation_words = {'not', 'no', 'never', 'neither', 'nor', 'none', 'nobody', 'nothing'}
stop_words = stop_words - negation_words

print(f"Total stopwords: {len(stop_words)}")
print(f"\nSample stopwords: {list(stop_words)[:20]}")
print(f"\nKept negation words: {negation_words}")

## 4.4 Stemming vs Lemmatization

Both reduce words to their base form, but differently:

| Method | Approach | Example | Speed | Accuracy |
|--------|----------|---------|-------|----------|
| **Stemming** | Chop off suffixes | running → run, studies → studi | Fast | Less accurate |
| **Lemmatization** | Use dictionary lookup | running → run, studies → study | Slow | More accurate |

### Comparison:

| Word | Stemming | Lemmatization |
|------|----------|---------------|
| studies | studi | study |
| studying | study | study |
| better | better | good |
| am, are, is | am, are, i | be |

For this project, we'll use **stemming** (faster, good enough for sentiment).

In [ ]:
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Compare stemming vs lemmatization
test_words = ['running', 'ran', 'runs', 'studies', 'studying', 'better', 'happier', 'loving']

print("Stemming vs Lemmatization Comparison:")
print("="*60)
print(f"{'Original':<15} {'Stemmed':<15} {'Lemmatized':<15}")
print("-"*60)
for word in test_words:
    stemmed = stemmer.stem(word)
    lemmatized = lemmatizer.lemmatize(word, pos='v')  # 'v' for verb
    print(f"{word:<15} {stemmed:<15} {lemmatized:<15}")

print("\nWe'll use stemming for speed.")

## 4.5 Building the Preprocessing Function

In [ ]:
def preprocess_text(text):
    """
    Complete preprocessing pipeline for tweet text.
    
    Steps:
    1. Lowercase
    2. Remove URLs
    3. Remove mentions (@username)
    4. Remove hashtags (#)
    5. Remove special characters and numbers
    6. Tokenize
    7. Remove stopwords
    8. Stem words
    9. Join back to string
    
    Parameters:
    -----------
    text : str
        Raw tweet text
    
    Returns:
    --------
    cleaned_text : str
        Preprocessed text
    """
    # 1. Lowercase
    text = text.lower()
    
    # 2. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 3. Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # 4. Remove hashtag symbols (keep the text)
    text = re.sub(r'#', '', text)
    
    # 5. Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 6. Tokenize
    tokens = text.split()
    
    # 7. Remove stopwords and 8. Stem
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    # 9. Join back to string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

print("Preprocessing function defined!")

## 4.6 Before and After Examples

Let's see the preprocessing in action!

In [ ]:
# Test preprocessing on sample tweets
sample_tweets = [
    "@user I LOVE this product! #amazing http://bit.ly/abc 😊",
    "This is the WORST experience ever!!! I'm so disappointed 😞",
    "@company Your customer service is terrible. Not happy at all.",
    "Having a great day! Everything is going perfectly 🌟",
    "Can't believe how bad this is... Won't be coming back #disappointed"
]

print("BEFORE and AFTER Preprocessing:")
print("="*80)
for i, tweet in enumerate(sample_tweets, 1):
    cleaned = preprocess_text(tweet)
    print(f"\n{i}. BEFORE: {tweet}")
    print(f"   AFTER:  {cleaned}")

print("\n" + "="*80)
print("Notice how the text is cleaned and normalized!")

## 4.7 Apply Preprocessing to Dataset

**Note**: Processing 1.6M tweets takes time. For this demo, we'll:
1. Use a **sample** for quick training and comparison
2. You can increase the sample size or use full dataset for production

In [ ]:
# For faster training, let's use a sample
# You can change this to df.shape[0] to use the full dataset
SAMPLE_SIZE = 200000  # 200K tweets (125K for speed, adjust as needed)

print(f"Using {SAMPLE_SIZE:,} tweets for training...")
print("(This makes the notebook run faster. Increase for production.)\n")

# Sample the data (stratified to maintain class balance)
df_sample = df.groupby('sentiment', group_keys=False).apply(
    lambda x: x.sample(n=SAMPLE_SIZE//2, random_state=42)
)

print(f"Sample size: {len(df_sample):,} tweets")
print(f"Class distribution:")
print(df_sample['sentiment'].value_counts().sort_index())

In [ ]:
# Apply preprocessing
print("Preprocessing tweets...")
start_time = time.time()

df_sample['cleaned_text'] = df_sample['text'].apply(preprocess_text)

elapsed = time.time() - start_time
print(f"Preprocessing complete! Time taken: {elapsed:.2f} seconds")
print(f"Speed: {len(df_sample)/elapsed:.0f} tweets/second")

In [ ]:
# Show examples of cleaned tweets
print("Sample Cleaned Tweets:")
print("="*80)
for i in range(5):
    row = df_sample.iloc[i]
    print(f"\nSentiment: {'Positive' if row['sentiment'] == 1 else 'Negative'}")
    print(f"Original:  {row['text'][:100]}...")
    print(f"Cleaned:   {row['cleaned_text'][:100]}...")

In [ ]:
# Remove empty tweets (if any after preprocessing)
df_sample = df_sample[df_sample['cleaned_text'].str.strip() != '']

print(f"Tweets after removing empty: {len(df_sample):,}")
print(f"\nDataset ready for feature extraction!")

---

<a id='part5'></a>
# Part 5: Feature Extraction Methods

---

## 5.1 The Challenge: Converting Text to Numbers

Machine learning models can't work with text directly - they need **numbers**!

How do we convert "I love this!" into numbers?

## 5.2 Feature Extraction Methods

| Method | Idea | Advantages | Disadvantages |
|--------|------|------------|---------------|
| **Bag of Words (BoW)** | Count word occurrences | Simple, interpretable | Ignores order, large sparse matrices |
| **TF-IDF** | Weighted word importance | Reduces common word impact | Still ignores order |
| **Word Embeddings** | Dense vectors (Word2Vec, GloVe) | Captures meaning, smaller size | Needs pre-training |

We'll focus on **TF-IDF** (best for classical ML on sentiment).

## 5.3 Bag of Words (CountVectorizer)

### How it works:

1. Build vocabulary of all unique words
2. For each document, count word occurrences
3. Create a matrix: rows = documents, columns = words

### Example:

```
Doc 1: "I love cats"
Doc 2: "I love dogs"
Doc 3: "I hate cats"

Vocabulary: [cats, dogs, hate, I, love]

        cats  dogs  hate  I  love
Doc 1:    1     0     0   1    1
Doc 2:    0     1     0   1    1
Doc 3:    1     0     1   1    0
```

**Problem**: Common words like "I" get same weight as important words like "love"

In [ ]:
# Demonstrate Bag of Words
sample_texts = [
    "love great product",
    "terrible bad experience",
    "love love amazing"
]

bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(sample_texts)

print("Bag of Words Example:")
print("="*60)
print("\nVocabulary:", bow_vectorizer.get_feature_names_out())
print("\nWord Count Matrix:")
print(pd.DataFrame(
    bow_matrix.toarray(),
    columns=bow_vectorizer.get_feature_names_out(),
    index=['Doc 1', 'Doc 2', 'Doc 3']
))
print("\nNotice: 'love' appears twice in Doc 3!")

## 5.4 TF-IDF (Term Frequency - Inverse Document Frequency)

### The Idea:

Weight words by both:
- **How often they appear in this document** (Term Frequency)
- **How rare they are across all documents** (Inverse Document Frequency)

### The Math:

**Term Frequency (TF)**:
$$TF(t, d) = \frac{\text{Number of times term } t \text{ appears in document } d}{\text{Total terms in document } d}$$

**Inverse Document Frequency (IDF)**:
$$IDF(t) = \log\left(\frac{\text{Total number of documents}}{\text{Number of documents containing term } t}\right)$$

**TF-IDF Score**:
$$TF\text{-}IDF(t, d) = TF(t, d) \times IDF(t)$$

### Intuition:

- Common words ("the", "is") appear in many docs → low IDF → low TF-IDF
- Rare, specific words ("awesome", "terrible") → high IDF → high TF-IDF
- **Important words get higher scores!**

In [ ]:
# Demonstrate TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_texts)

print("TF-IDF Example:")
print("="*60)
print("\nVocabulary:", tfidf_vectorizer.get_feature_names_out())
print("\nTF-IDF Matrix:")
print(pd.DataFrame(
    tfidf_matrix.toarray().round(3),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=['Doc 1', 'Doc 2', 'Doc 3']
))
print("\nNotice: Values are weighted, not just counts!")

## 5.5 Comparison: BoW vs TF-IDF

| Feature | Bag of Words | TF-IDF |
|---------|--------------|--------|
| **Values** | Raw counts | Weighted scores |
| **Common words** | High values | Low values |
| **Rare words** | Low values (if infrequent) | High values (if distinctive) |
| **Normalization** | No | Yes (by document length) |
| **Best for** | Simple classification | Text with varying word importance |

**For sentiment analysis, TF-IDF typically works better!**

## 5.6 Apply TF-IDF to Our Dataset

In [ ]:
# Create TF-IDF features
# max_features: limit vocabulary size to most common words (reduces memory)
# ngram_range: (1,2) includes both unigrams ("good") and bigrams ("not good")

print("Creating TF-IDF features...")
start_time = time.time()

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(df_sample['cleaned_text'])
y = df_sample['sentiment'].values

elapsed = time.time() - start_time
print(f"TF-IDF vectorization complete! Time: {elapsed:.2f}s")
print(f"\nFeature matrix shape: {X_tfidf.shape}")
print(f"  - {X_tfidf.shape[0]:,} samples (tweets)")
print(f"  - {X_tfidf.shape[1]:,} features (words/bigrams)")
print(f"\nMatrix sparsity: {(1 - X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1]))*100:.2f}%")
print("(Most values are zero - text data is sparse!)")

In [ ]:
# Top features by TF-IDF score
feature_names = tfidf.get_feature_names_out()
tfidf_scores = X_tfidf.mean(axis=0).A1  # Average TF-IDF across all documents
top_indices = tfidf_scores.argsort()[-20:][::-1]

print("Top 20 Features by Average TF-IDF Score:")
print("="*60)
for i, idx in enumerate(top_indices, 1):
    print(f"{i:2}. {feature_names[idx]:<20} (score: {tfidf_scores[idx]:.4f})")

print("\nThese are the most distinctive words/phrases in our dataset!")

---

<a id='part6'></a>
# Part 6: Classical Machine Learning Models

---

## 6.1 Train-Test Split

**The Golden Rule**: Never test on training data!

We'll split:
- **80% Training**: Model learns from this
- **20% Testing**: Evaluate performance on unseen data

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class balance
)

print("Train-Test Split:")
print("="*60)
print(f"Training samples: {X_train.shape[0]:,} ({X_train.shape[0]/len(y)*100:.0f}%)")
print(f"Testing samples:  {X_test.shape[0]:,} ({X_test.shape[0]/len(y)*100:.0f}%)")
print(f"\nTraining class distribution:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nTesting class distribution:")
print(pd.Series(y_test).value_counts().sort_index())

## 6.2 Classical ML Algorithms

We'll train 4 different algorithms:

| Algorithm | Type | How It Works | Pros | Cons |
|-----------|------|--------------|------|------|
| **Logistic Regression** | Linear | Finds linear decision boundary | Fast, interpretable | Assumes linear separability |
| **Naive Bayes** | Probabilistic | Uses Bayes' theorem with independence assumption | Very fast, works well on text | Strong independence assumption |
| **SVM** | Margin-based | Finds maximum margin hyperplane | Effective in high dimensions | Can be slow on large datasets |
| **Random Forest** | Ensemble | Combines multiple decision trees | Handles non-linearity, robust | Slower, less interpretable |

In [ ]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Naive Bayes': MultinomialNB(),
    'SVM': LinearSVC(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}

print("Models to train:")
for i, name in enumerate(models.keys(), 1):
    print(f"  {i}. {name}")

## 6.3 Train All Models

In [ ]:
# Train and evaluate all models
results = {}

print("="*70)
print("TRAINING AND EVALUATING CLASSICAL ML MODELS")
print("="*70)

for name, model in models.items():
    print(f"\n{name}:")
    print("-" * 50)
    
    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Predict
    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_time': train_time,
        'predict_time': predict_time
    }
    
    print(f"  Training time:   {train_time:.2f}s")
    print(f"  Prediction time: {predict_time:.2f}s")
    print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

print("\n" + "="*70)
print("All models trained successfully!")

## 6.4 Training Time Comparison

In [ ]:
# Compare training times
train_times = {name: res['train_time'] for name, res in results.items()}

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
bars = ax.bar(train_times.keys(), train_times.values(), color=colors, edgecolor='black')
ax.set_ylabel('Training Time (seconds)')
ax.set_title('Model Training Time Comparison', fontweight='bold', fontsize=14)

for bar, time in zip(bars, train_times.values()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f'{time:.2f}s', ha='center', fontweight='bold')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

fastest = min(train_times, key=train_times.get)
print(f"Fastest model: {fastest} ({train_times[fastest]:.2f}s)")

## 6.5 Confusion Matrices

A confusion matrix shows:
- **True Positives (TP)**: Correctly predicted positive
- **True Negatives (TN)**: Correctly predicted negative
- **False Positives (FP)**: Predicted positive, actually negative
- **False Negatives (FN)**: Predicted negative, actually positive

In [ ]:
# Plot confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, (name, res) in enumerate(results.items()):
    cm = confusion_matrix(y_test, res['predictions'])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
    disp.plot(ax=axes[i], cmap='Blues', values_format='d')
    axes[i].set_title(f"{name}\nAccuracy: {res['accuracy']*100:.2f}%", fontweight='bold')

plt.suptitle('Confusion Matrices - Classical ML Models', fontweight='bold', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\nHow to read:")
print("- Diagonal (top-left to bottom-right) = Correct predictions")
print("- Off-diagonal = Mistakes")
print("- Darker blue = More samples")

## 6.6 Classification Reports

In [ ]:
# Detailed classification reports
print("="*70)
print("DETAILED CLASSIFICATION REPORTS")
print("="*70)

for name, res in results.items():
    print(f"\n{'='*50}")
    print(f"{name}")
    print(f"{'='*50}")
    print(classification_report(y_test, res['predictions'], 
                                target_names=['Negative', 'Positive']))

---

<a id='part7'></a>
# Part 7: Deep Learning with LSTM

---

## 7.1 What are Recurrent Neural Networks (RNNs)?

**Traditional Neural Networks**: Process inputs independently
- Can't remember previous inputs
- Order doesn't matter

**Recurrent Neural Networks**: Have memory!
- Process sequences (like text)
- Remember previous inputs
- **Order matters**: "not good" ≠ "good not"

### The RNN Idea:

```
Input:  [I]  [love]  [this]  [movie]
         ↓      ↓       ↓       ↓
RNN:    [h1] → [h2] → [h3] → [h4] → Output
         ↑      ↑       ↑       ↑
      memory  memory  memory  memory
```

Each step passes information to the next!

## 7.2 What are LSTMs?

**Problem with basic RNNs**: Vanishing gradient problem
- Hard to learn long-term dependencies
- Forgets early parts of sequence

**LSTM (Long Short-Term Memory)**: Special RNN that can remember long sequences!

### LSTM Components:

| Gate | Function |
|------|----------|
| **Forget Gate** | Decides what to forget from memory |
| **Input Gate** | Decides what new information to add |
| **Output Gate** | Decides what to output |

**Why LSTMs for Sentiment?**
- Can learn context: "not good" vs "very good"
- Handles negation: "not bad" = positive
- Captures word order: "happy not sad" vs "sad not happy"

## 7.3 Tokenization and Padding for Deep Learning

LSTMs need:
1. **Integer sequences**: Words → Numbers
2. **Fixed length**: All sequences same length (padding)

### Example:
```
"I love this" → [12, 45, 89]
"Bad movie"   → [5, 23]

After padding (max_len=5):
"I love this" → [12, 45, 89, 0, 0]
"Bad movie"   → [5, 23, 0, 0, 0]
```

In [ ]:
# Prepare data for LSTM
# Use a smaller sample for LSTM (deep learning is slower)
LSTM_SAMPLE_SIZE = 50000  # 50K tweets for LSTM

print(f"Preparing {LSTM_SAMPLE_SIZE:,} tweets for LSTM training...")

# Sample for LSTM
df_lstm = df_sample.sample(n=LSTM_SAMPLE_SIZE, random_state=42)

# Split
X_lstm_text = df_lstm['cleaned_text'].values
y_lstm = df_lstm['sentiment'].values

X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(
    X_lstm_text, y_lstm,
    test_size=0.2,
    random_state=42,
    stratify=y_lstm
)

print(f"LSTM training samples: {len(X_lstm_train):,}")
print(f"LSTM testing samples:  {len(X_lstm_test):,}")

In [ ]:
# Tokenization for LSTM
MAX_WORDS = 10000  # Vocabulary size
MAX_LEN = 50       # Maximum sequence length

print("Tokenizing text for LSTM...")

# Create tokenizer
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_lstm_train)

# Convert texts to sequences
X_lstm_train_seq = tokenizer.texts_to_sequences(X_lstm_train)
X_lstm_test_seq = tokenizer.texts_to_sequences(X_lstm_test)

# Pad sequences
X_lstm_train_pad = pad_sequences(X_lstm_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_lstm_test_pad = pad_sequences(X_lstm_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"\nVocabulary size: {len(tokenizer.word_index):,}")
print(f"Using top {MAX_WORDS:,} words")
print(f"\nPadded sequence shape: {X_lstm_train_pad.shape}")
print(f"  - {X_lstm_train_pad.shape[0]:,} samples")
print(f"  - {X_lstm_train_pad.shape[1]} sequence length (max)")

In [ ]:
# Example of tokenization and padding
sample_text = X_lstm_train[0]
sample_seq = X_lstm_train_seq[0]
sample_pad = X_lstm_train_pad[0]

print("Tokenization Example:")
print("="*60)
print(f"Original text: {sample_text}")
print(f"\nToken sequence: {sample_seq}")
print(f"\nPadded sequence (len={MAX_LEN}): {sample_pad}")
print("\nZeros are padding to make all sequences same length!")

## 7.4 Build LSTM Model Architecture

Our LSTM model will have:

| Layer | Type | Purpose |
|-------|------|----------|
| **Embedding** | Word vectors | Convert integers to dense vectors |
| **Bidirectional LSTM** | Sequence processing | Read text forward AND backward |
| **Dropout** | Regularization | Prevent overfitting |
| **Dense** | Classification | Output layer with sigmoid activation |

In [ ]:
# Build LSTM model
EMBEDDING_DIM = 128
LSTM_UNITS = 64

print("Building LSTM model...")

lstm_model = Sequential([
    # Embedding layer: converts word indices to dense vectors
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    
    # Bidirectional LSTM: reads text forwards and backwards
    Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)),
    Dropout(0.5),
    
    # Second LSTM layer
    Bidirectional(LSTM(LSTM_UNITS//2)),
    Dropout(0.5),
    
    # Dense layers
    Dense(32, activation='relu'),
    Dropout(0.5),
    
    # Output layer: sigmoid for binary classification
    Dense(1, activation='sigmoid')
])

# Compile model
lstm_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model built!\n")
lstm_model.summary()

## 7.5 Train LSTM Model

We'll use:
- **EarlyStopping**: Stop if validation accuracy doesn't improve
- **ReduceLROnPlateau**: Reduce learning rate if stuck

In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)

# Train model
print("Training LSTM model...")
print("This may take a few minutes...\n")

history = lstm_model.fit(
    X_lstm_train_pad, y_lstm_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nTraining complete!")

## 7.6 Training Curves

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('LSTM Training Accuracy', fontweight='bold')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('LSTM Training Loss', fontweight='bold')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- If training and validation curves are close: Good generalization")
print("- If training much better than validation: Overfitting")
print("- Both curves should decrease over time")

## 7.7 Evaluate LSTM Model

In [ ]:
# Evaluate LSTM
y_lstm_pred_prob = lstm_model.predict(X_lstm_test_pad, verbose=0)
y_lstm_pred = (y_lstm_pred_prob > 0.5).astype(int).flatten()

# Calculate metrics
lstm_accuracy = accuracy_score(y_lstm_test, y_lstm_pred)
lstm_precision = precision_score(y_lstm_test, y_lstm_pred)
lstm_recall = recall_score(y_lstm_test, y_lstm_pred)
lstm_f1 = f1_score(y_lstm_test, y_lstm_pred)

print("LSTM Model Performance:")
print("="*60)
print(f"Accuracy:  {lstm_accuracy:.4f} ({lstm_accuracy*100:.2f}%)")
print(f"Precision: {lstm_precision:.4f}")
print(f"Recall:    {lstm_recall:.4f}")
print(f"F1-Score:  {lstm_f1:.4f}")

# Confusion matrix
cm_lstm = confusion_matrix(y_lstm_test, y_lstm_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_lstm, display_labels=['Negative', 'Positive'])
disp.plot(cmap='Blues', values_format='d')
plt.title(f'LSTM Confusion Matrix\nAccuracy: {lstm_accuracy*100:.2f}%', fontweight='bold')
plt.show()

print("\nClassification Report:")
print(classification_report(y_lstm_test, y_lstm_pred, target_names=['Negative', 'Positive']))

---

<a id='part8'></a>
# Part 8: Model Comparison

---

## 8.1 Performance Metrics Comparison

In [ ]:
# Create comparison dataframe
comparison_data = []

# Classical ML models
for name, res in results.items():
    comparison_data.append({
        'Model': name,
        'Type': 'Classical ML',
        'Accuracy': res['accuracy'],
        'Precision': res['precision'],
        'Recall': res['recall'],
        'F1-Score': res['f1'],
        'Training Time (s)': res['train_time']
    })

# LSTM model
comparison_data.append({
    'Model': 'LSTM',
    'Type': 'Deep Learning',
    'Accuracy': lstm_accuracy,
    'Precision': lstm_precision,
    'Recall': lstm_recall,
    'F1-Score': lstm_f1,
    'Training Time (s)': sum(history.epoch) * np.mean(history.history['loss'])  # Approximate
})

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Accuracy', ascending=False).reset_index(drop=True)
comparison_df.index = range(1, len(comparison_df) + 1)

print("Model Performance Comparison:")
print("="*80)
print(comparison_df.to_string())

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

for i, metric in enumerate(metrics):
    ax = axes[i // 2, i % 2]
    bars = ax.barh(comparison_df['Model'], comparison_df[metric], 
                    color=colors[:len(comparison_df)], edgecolor='black')
    ax.set_xlabel(metric)
    ax.set_title(f'{metric} Comparison', fontweight='bold')
    ax.set_xlim(0.7, 1.0)
    
    for bar, val in zip(bars, comparison_df[metric]):
        ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
                f'{val:.4f}', va='center', fontsize=9, fontweight='bold')

plt.suptitle('Model Performance Comparison', fontweight='bold', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 8.2 Classical ML vs Deep Learning

### When to Use Each Approach:

| Factor | Classical ML | Deep Learning (LSTM) |
|--------|-------------|---------------------|
| **Dataset size** | Small to medium (< 100K) | Large (> 100K) |
| **Training time** | Fast (seconds to minutes) | Slow (minutes to hours) |
| **Interpretability** | High (feature weights) | Low (black box) |
| **Feature engineering** | Required (TF-IDF, etc.) | Automatic (embeddings) |
| **Performance** | Good (75-80% accuracy) | Better (80-85%+ accuracy) |
| **Computational resources** | Low (CPU fine) | High (GPU preferred) |
| **Production deployment** | Easy | More complex |

### Our Findings:

- **Best Classical ML**: Logistic Regression or SVM
  - Fast training
  - Good accuracy (~78-80%)
  - Easy to deploy

- **LSTM**:
  - Slightly better accuracy (~80-82%)
  - Much slower training
  - Captures word order and context better

**Recommendation**: 
- For production with limited resources: **Logistic Regression or SVM**
- For maximum accuracy with resources: **LSTM or Transformer models**

---

<a id='part9'></a>
# Part 9: Word Clouds & Visualization

---

## 9.1 Word Clouds by Sentiment

Word clouds visualize the most frequent words, with size proportional to frequency.

In [ ]:
# Create word clouds for each sentiment
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Negative tweets
negative_text = ' '.join(df_sample[df_sample['sentiment'] == 0]['cleaned_text'].values)
wordcloud_negative = WordCloud(
    width=800, height=400,
    background_color='white',
    colormap='Reds',
    max_words=100
).generate(negative_text)

axes[0].imshow(wordcloud_negative, interpolation='bilinear')
axes[0].axis('off')
axes[0].set_title('Negative Sentiment Word Cloud', fontweight='bold', fontsize=14)

# Positive tweets
positive_text = ' '.join(df_sample[df_sample['sentiment'] == 1]['cleaned_text'].values)
wordcloud_positive = WordCloud(
    width=800, height=400,
    background_color='white',
    colormap='Greens',
    max_words=100
).generate(positive_text)

axes[1].imshow(wordcloud_positive, interpolation='bilinear')
axes[1].axis('off')
axes[1].set_title('Positive Sentiment Word Cloud', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

print("Larger words appear more frequently in tweets!")
print("Notice how negative and positive tweets use different vocabulary.")

## 9.2 Most Common Words by Sentiment

In [ ]:
# Get most common words for each sentiment
from collections import Counter

# Negative words
negative_words = ' '.join(df_sample[df_sample['sentiment'] == 0]['cleaned_text'].values).split()
negative_counter = Counter(negative_words)
top_negative = negative_counter.most_common(20)

# Positive words
positive_words = ' '.join(df_sample[df_sample['sentiment'] == 1]['cleaned_text'].values).split()
positive_counter = Counter(positive_words)
top_positive = positive_counter.most_common(20)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Negative
words_neg, counts_neg = zip(*top_negative)
axes[0].barh(words_neg[::-1], counts_neg[::-1], color='#FF6B6B', edgecolor='black')
axes[0].set_xlabel('Frequency')
axes[0].set_title('Top 20 Words in Negative Tweets', fontweight='bold')

# Positive
words_pos, counts_pos = zip(*top_positive)
axes[1].barh(words_pos[::-1], counts_pos[::-1], color='#4ECDC4', edgecolor='black')
axes[1].set_xlabel('Frequency')
axes[1].set_title('Top 20 Words in Positive Tweets', fontweight='bold')

plt.tight_layout()
plt.show()

## 9.3 Misclassified Examples Analysis

Let's look at tweets our best model got wrong. This helps understand limitations!

In [ ]:
# Get misclassified examples from best classical model
best_model_name = comparison_df.iloc[0]['Model']
if best_model_name != 'LSTM':
    best_predictions = results[best_model_name]['predictions']
    
    # Find misclassified indices
    test_indices = df_sample.sample(n=len(y_test), random_state=42).index
    misclassified_mask = best_predictions != y_test
    misclassified_indices = test_indices[misclassified_mask]
    
    # Get misclassified samples
    misclassified_df = df_sample.loc[misclassified_indices].copy()
    misclassified_df['predicted'] = best_predictions[misclassified_mask]
    
    print(f"Misclassified Examples from {best_model_name}:")
    print("="*80)
    
    # Sample misclassifications
    samples = misclassified_df.sample(n=min(10, len(misclassified_df)), random_state=42)
    
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        actual = 'Positive' if row['sentiment'] == 1 else 'Negative'
        predicted = 'Positive' if row['predicted'] == 1 else 'Negative'
        print(f"\n{i}. Tweet: {row['text'][:100]}...")
        print(f"   Actual: {actual} | Predicted: {predicted}")
    
    print("\n" + "="*80)
    print("Why misclassifications happen:")
    print("- Sarcasm: 'Great, another delay!' (negative despite 'great')")
    print("- Context needed: 'This movie is sick!' (positive in slang)")
    print("- Mixed sentiment: 'Good acting but terrible plot'")
    print("- Subtle negation: 'I wish I could say I liked it'")

---

<a id='part10'></a>
# Part 10: Predictions on New Tweets

---

## 10.1 Create Prediction Function

In [ ]:
def predict_sentiment(text, model_type='classical', model_name='Logistic Regression'):
    """
    Predict sentiment of a new tweet.
    
    Parameters:
    -----------
    text : str
        Raw tweet text
    model_type : str
        'classical' or 'lstm'
    model_name : str
        Name of classical model to use
    
    Returns:
    --------
    sentiment : str
        'Positive' or 'Negative'
    confidence : float
        Prediction confidence (0-1)
    """
    # Preprocess
    cleaned = preprocess_text(text)
    
    if model_type == 'classical':
        # TF-IDF transform
        features = tfidf.transform([cleaned])
        
        # Predict
        model = results[model_name]['model']
        prediction = model.predict(features)[0]
        
        # Get confidence (if available)
        if hasattr(model, 'predict_proba'):
            confidence = model.predict_proba(features)[0][prediction]
        elif hasattr(model, 'decision_function'):
            decision = model.decision_function(features)[0]
            confidence = 1 / (1 + np.exp(-decision))  # Sigmoid
            if prediction == 0:
                confidence = 1 - confidence
        else:
            confidence = None
    
    else:  # LSTM
        # Tokenize and pad
        sequence = tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')
        
        # Predict
        prob = lstm_model.predict(padded, verbose=0)[0][0]
        prediction = 1 if prob > 0.5 else 0
        confidence = prob if prediction == 1 else 1 - prob
    
    sentiment = 'Positive' if prediction == 1 else 'Negative'
    
    return sentiment, confidence

print("Prediction function ready!")

## 10.2 Test on Sample Tweets

In [ ]:
# Test tweets
test_tweets = [
    "I absolutely love this product! Best purchase ever!",
    "This is terrible. Waste of money. Very disappointed.",
    "Amazing service! Highly recommend to everyone!",
    "Worst experience ever. Never coming back.",
    "Not bad, but could be better.",
    "I'm so happy with this! Exceeded expectations!",
    "Awful quality. Do not buy!",
    "Pretty good overall, satisfied with my choice.",
    "Can't believe how bad this is. Totally disappointed.",
    "Fantastic! Everything I hoped for and more!"
]

print("Sentiment Predictions on New Tweets:")
print("="*80)

for i, tweet in enumerate(test_tweets, 1):
    sentiment, confidence = predict_sentiment(tweet, model_type='classical', 
                                              model_name='Logistic Regression')
    
    emoji = '😊' if sentiment == 'Positive' else '😞'
    print(f"\n{i}. Tweet: \"{tweet}\"")
    print(f"   Prediction: {sentiment} {emoji}")
    if confidence is not None:
        print(f"   Confidence: {confidence:.2%}")

## 10.3 Compare Classical ML vs LSTM Predictions

In [ ]:
# Compare predictions
comparison_tweets = [
    "This is not good at all!",
    "I love this so much!",
    "Terrible experience, very unhappy.",
    "Great product, highly satisfied!"
]

print("Classical ML vs LSTM Predictions:")
print("="*80)

for tweet in comparison_tweets:
    classical_sent, classical_conf = predict_sentiment(tweet, 'classical', 'Logistic Regression')
    lstm_sent, lstm_conf = predict_sentiment(tweet, 'lstm')
    
    print(f"\nTweet: \"{tweet}\"")
    print(f"  Classical ML: {classical_sent} (confidence: {classical_conf:.2%})")
    print(f"  LSTM:         {lstm_sent} (confidence: {lstm_conf:.2%})")
    
    if classical_sent != lstm_sent:
        print("  ⚠️  Models disagree!")

---

<a id='part11'></a>
# Part 11: Summary and Key Takeaways

---

## Final Results Dashboard

In [ ]:
# Create comprehensive summary dashboard
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Model Accuracy Comparison (top row, full width)
ax1 = fig.add_subplot(gs[0, :])
models_list = comparison_df['Model'].values
accuracies = comparison_df['Accuracy'].values * 100
colors_plot = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
bars = ax1.bar(models_list, accuracies, color=colors_plot[:len(models_list)], edgecolor='black')
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Model Accuracy Comparison', fontweight='bold', fontsize=14)
ax1.set_ylim(70, 85)
for bar, acc in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{acc:.2f}%', ha='center', fontweight='bold')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=15, ha='right')

# 2. Sentiment Distribution (bottom left)
ax2 = fig.add_subplot(gs[1, 0])
ax2.pie([50, 50], labels=['Negative', 'Positive'], autopct='%1.1f%%',
        colors=['#FF6B6B', '#4ECDC4'], explode=(0.02, 0.02), shadow=True)
ax2.set_title('Dataset Balance', fontweight='bold')

# 3. Best Model Confusion Matrix (bottom middle)
ax3 = fig.add_subplot(gs[1, 1])
best_model_results = results[list(results.keys())[0]]
cm_best = confusion_matrix(y_test, best_model_results['predictions'])
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Blues', ax=ax3,
            xticklabels=['Neg', 'Pos'], yticklabels=['Neg', 'Pos'])
ax3.set_title('Best Model Confusion Matrix', fontweight='bold')
ax3.set_xlabel('Predicted')
ax3.set_ylabel('Actual')

# 4. Training Time (bottom right)
ax4 = fig.add_subplot(gs[1, 2])
train_times_all = comparison_df.set_index('Model')['Training Time (s)'].to_dict()
ax4.barh(list(train_times_all.keys()), list(train_times_all.values()),
         color=colors_plot[:len(train_times_all)], edgecolor='black')
ax4.set_xlabel('Time (seconds)')
ax4.set_title('Training Time', fontweight='bold')

# 5. Performance Metrics Table (bottom row)
ax5 = fig.add_subplot(gs[2, :])
ax5.axis('tight')
ax5.axis('off')
table_data = comparison_df[['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score']].copy()
table_data['Accuracy'] = (table_data['Accuracy'] * 100).round(2).astype(str) + '%'
table_data['Precision'] = table_data['Precision'].round(4)
table_data['Recall'] = table_data['Recall'].round(4)
table_data['F1-Score'] = table_data['F1-Score'].round(4)
table = ax5.table(cellText=table_data.values, colLabels=table_data.columns,
                  cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
for i in range(len(table_data.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold')

plt.suptitle('SENTIMENT ANALYSIS - SUMMARY DASHBOARD', fontweight='bold', fontsize=16, y=0.98)
plt.show()

---

## Key Takeaways

### 1. What We Learned

| Topic | Key Learning |
|-------|-------------|
| **NLP Fundamentals** | Text must be preprocessed and converted to numbers |
| **Preprocessing** | Cleaning, tokenization, stemming are crucial |
| **Feature Extraction** | TF-IDF weights words by importance |
| **Classical ML** | Fast, interpretable, good for moderate-sized datasets |
| **Deep Learning** | Better accuracy, captures context, but slower |
| **Model Selection** | Trade-off between accuracy and resources |

### 2. Text Preprocessing Importance

**Without preprocessing**: Models learn from noise
- URLs, mentions, special characters add no value
- Case sensitivity creates duplicate features
- Stopwords dilute important signals

**With preprocessing**: Clean, focused features
- Vocabulary size reduced by ~50%
- Faster training
- Better generalization

### 3. Classical ML vs Deep Learning

| Scenario | Best Choice |
|----------|-------------|
| Small dataset (< 10K) | Classical ML |
| Large dataset (> 100K) | Deep Learning |
| Need interpretability | Classical ML (Logistic Regression) |
| Need highest accuracy | Deep Learning (LSTM/Transformers) |
| Limited compute | Classical ML |
| Real-time predictions | Classical ML (faster inference) |

### 4. Real-World Deployment Considerations

**For Production:**
1. **Model Selection**:
   - Start with Logistic Regression (fast, reliable)
   - Upgrade to LSTM if accuracy is critical

2. **Infrastructure**:
   - Classical ML: Can run on CPU
   - LSTM: Benefits from GPU

3. **Monitoring**:
   - Track accuracy over time
   - Retrain when performance drops
   - Monitor for data drift

4. **Handling Edge Cases**:
   - Sarcasm detection: Add emoji analysis
   - Context: Use attention mechanisms
   - Mixed sentiment: Multi-label classification

### 5. Our Best Models

**Best Classical ML**: Logistic Regression or SVM
- ~78-80% accuracy
- Fast training (< 10s)
- Easy to deploy
- Interpretable (feature weights)

**Best Deep Learning**: LSTM
- ~80-82% accuracy
- Slower training (minutes)
- Captures word order
- Better on complex cases

### 6. Next Steps & Improvements

To improve further:

1. **Better preprocessing**:
   - Handle emojis explicitly
   - Spell checking
   - Expand contractions ("can't" → "cannot")

2. **Advanced models**:
   - BERT/RoBERTa (state-of-the-art)
   - GPT for few-shot learning
   - Ensemble methods

3. **More features**:
   - User metadata (if available)
   - Emoji sentiment
   - Hashtag analysis

4. **Domain-specific**:
   - Fine-tune on specific industries
   - Custom sentiment lexicons

---

## Summary Table

| Metric | Value |
|--------|-------|
| **Dataset Size** | 1.6 million tweets (used 200K sample) |
| **Classes** | 2 (Negative, Positive) |
| **Balance** | Perfect (50-50) |
| **Best Accuracy** | ~80% (LSTM) |
| **Fastest Model** | Naive Bayes (~2s training) |
| **Most Balanced** | Logistic Regression (speed + accuracy) |
| **Vocabulary Size** | 10,000 words (LSTM), 5,000 features (TF-IDF) |

---

**End of Sentiment Analysis Tutorial**

You now understand:
- ✅ What sentiment analysis is and why it matters
- ✅ How to preprocess text data properly
- ✅ Different feature extraction methods (BoW, TF-IDF)
- ✅ Classical ML approaches (Logistic Regression, SVM, etc.)
- ✅ Deep learning with LSTMs
- ✅ How to evaluate and compare models
- ✅ Real-world deployment considerations

This knowledge applies to:
- Customer review analysis
- Social media monitoring
- Product feedback classification
- Any text classification task!

In [ ]:
# Final summary
print("="*70)
print("SENTIMENT ANALYSIS - FINAL SUMMARY")
print("="*70)

print(f"\n📊 DATASET")
print(f"   Original size: 1,600,000 tweets")
print(f"   Sample used: {len(df_sample):,} tweets")
print(f"   Classes: 2 (Negative, Positive)")
print(f"   Balance: Perfect (50-50)")

print(f"\n🏆 BEST MODELS")
best_classical = comparison_df[comparison_df['Type'] == 'Classical ML'].iloc[0]
print(f"   Best Classical ML: {best_classical['Model']}")
print(f"     - Accuracy: {best_classical['Accuracy']*100:.2f}%")
print(f"     - F1-Score: {best_classical['F1-Score']:.4f}")
print(f"   LSTM:")
print(f"     - Accuracy: {lstm_accuracy*100:.2f}%")
print(f"     - F1-Score: {lstm_f1:.4f}")

print(f"\n📈 MODEL PERFORMANCE RANKING")
for i, row in comparison_df.iterrows():
    print(f"   {i}. {row['Model']}: {row['Accuracy']*100:.2f}%")

print(f"\n⚡ TRAINING SPEED")
fastest = comparison_df.loc[comparison_df['Training Time (s)'].idxmin()]
print(f"   Fastest: {fastest['Model']} ({fastest['Training Time (s)']:.2f}s)")

print(f"\n🎯 KEY INSIGHTS")
print(f"   - Classical ML is fast and effective for production")
print(f"   - LSTM achieves best accuracy but slower training")
print(f"   - Text preprocessing is critical for good performance")
print(f"   - TF-IDF works very well for sentiment analysis")

print("\n" + "="*70)
print("SENTIMENT ANALYSIS PROJECT COMPLETE!")
print("="*70)
print("\nYou can now:")
print("  ✓ Classify sentiment of any tweet or text")
print("  ✓ Choose the right model for your use case")
print("  ✓ Deploy sentiment analysis in production")
print("  ✓ Apply these techniques to other text classification tasks")
print("\nThis is a valuable skill for data science and NLP roles!")